In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
data = pd.read_csv("data_small.csv")

In [3]:
data.head()

,context,tag,words
0,"读者故事：“婆婆陪睡，我成了保姆” "" 茫茫人海中，为防大家走失，请大家 点击上方...",0,读者 故事 婆婆 陪 睡 我成 保姆 茫茫人海 中 为防 走失 请 点击 上方 婆媳关系 婚...
1,剖宫产期间产妇低体温的预测因素 背景与目的 虽然围术期低体温可增加产妇发病率，但少...,1,剖宫产 期间 产妇 低 体温 预测 因素 背景 目的 围术期 低 体温 增加 产妇 发病率 ...
2,挨过这波雨雪和降温，温暖和阳光就在前方！还要熬几天？ 这几天出门的小伙伴 抬头望天 都要...,1,挨过 这波 雨雪 降温 温暖 阳光 前方 熬 几天 几天 出门 小伙伴 抬头 望天 叹一口气...
3,杨立新：我不喝豆汁，太馊了！陈佩斯回答7个字让网友坐不住了！ 杨立新：我不喝豆汁，太馊了！陈...,1,杨立新 喝 豆汁 太 馊 陈佩斯 回答 字 网友 坐不住 杨立新 喝 豆汁 太 馊 陈佩斯 ...
4,上汽大众2018年销量继续领跑国内乘用车市场，背后的实力值得深究 2018年的车市格外严峻...,1,上汽 大众 2018 年销量 领跑 国内 乘用车 市场 背后 实力 值得 深究 2018 车...


In [4]:
stopwords = []
with open('stopwords.txt', "r", encoding='utf-8') as f:
    line = f.readline()
    while line:
        stopwords.append(line[:-1])
        line = f.readline()
stopwords = set(stopwords)

In [5]:
import jieba
document = []    
for i in range(len(data['context'])):
    seg_list = list(jieba.cut(data['context'][i]))
    sentence = ["".join(w) for w in seg_list if w not in stopwords]
    document.append(sentence)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.852 seconds.
Prefix dict has been built succesfully.


In [6]:
import gensim

model = gensim.models.Word2Vec(document, min_count=10, size=100) # min_count忽略词频低于10的词，默认值是5， size词向量维度，默认为100

In [7]:
model['故事'] # 词向量

array([-0.11778619,  0.24294573, -0.30936217, -0.25150734,  0.6710721 ,
       -0.4556564 ,  0.12344617,  0.49587384, -0.28393254,  0.01561437,
       -0.65755683, -0.28685528, -0.08796494,  0.39799744, -0.6603641 ,
        0.47582784, -0.3371661 , -0.07821414, -0.4025913 , -0.16783626,
       -0.44868666,  0.06887891,  0.2567606 , -0.04263752, -0.14793076,
       -0.02702715,  0.07139065, -0.00514326,  0.23485598, -0.40072027,
        0.08923026, -0.06745334,  0.15784648, -0.26475647, -0.3848933 ,
        0.8535356 , -0.56463015,  0.38508487,  0.43449122, -0.0248362 ,
       -0.052674  ,  0.06086592,  0.3538074 , -1.1369187 , -0.294113  ,
        0.35291755, -0.2500306 , -0.0045505 , -0.01700079, -0.05367097,
       -0.17493391, -0.4735483 ,  0.31075847, -0.40381458,  0.08349467,
       -0.68541074, -0.075026  ,  0.31664196,  0.1499828 ,  0.07543008,
        0.36977926, -0.11807282, -0.23606968, -0.2159341 , -0.91194487,
       -0.07460345,  0.38509822, -0.4825328 , -0.7250072 ,  0.00

In [8]:
# 计算最相近的topn个词
model.most_similar(['故事'], topn=10)

[('一位', 0.9999282360076904),
 ('最终', 0.9999151229858398),
 ('建议', 0.9999114274978638),
 ('希望', 0.9999090433120728),
 ('地方', 0.9999085068702698),
 ('更是', 0.9999076128005981),
 ('电影', 0.9999070167541504),
 ('前', 0.9999065399169922),
 ('历史', 0.9999058246612549),
 ('两个', 0.9999045133590698)]

In [9]:
# 计算相似概率，即词向量的cosine值
model.similarity('女人', '男人')

0.9999026

In [10]:
# 计算两句话的相似性
list1 = ['选择', '两个', '故事'] 
list2 = ['选择', '两个'] 
list_sim = model.n_similarity(list1, list2)
print(list_sim)

0.9999917


In [11]:
# 发现不匹配的词，也就是不同类的词
model.doesnt_match('选择 两个 故事'.split())

'故事'

In [12]:
model.save("word2vec.model") # 保存模型

In [13]:
model = gensim.models.Word2Vec.load('word2vec.model') # 导入模型

In [ ]:
model = gensim.models.Word2Vec.load('word2vec.model') 
model.train(more_sentences) # 用新数据追加训练

In [14]:
from gensim.models.word2vec import Word2Vec

# 该构造函数执行了三个步骤：
# 1.建立一个空的模型对象，
# 2.遍历一次语料库建立词典，
# 3.第二次遍历语料库建立神经网络模型
model = Word2Vec()
model.build_vocab(document)
model.train(document, total_examples=model.corpus_count, epochs=model.iter)

(479748, 704335)